# E6998 – HW2 Solution
### Student Columbia Uni: yg2499

#### Note: The code is running under Python 3.x version

### Step A: Train a model to classify the same topics from class: fishing, hiking, machinelearning and mathematics using the train.py script. 

In [7]:
import pandas as pd
import numpy as np
from sklearn import feature_extraction
from scipy import stats
from sklearn import decomposition,linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,GradientBoostingClassifier
from sklearn.manifold import Isomap
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import RidgeClassifier,Lasso,SGDClassifier,LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,f1_score,hamming_loss
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score,precision_recall_curve,f1_score
from sklearn.svm import SVC, LinearSVC
import sklearn
import nltk
import os
import re
from nltk.corpus import stopwords,wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from sklearn import decomposition
import time
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
from sklearn.manifold import Isomap
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from collections import Counter
#from multiscorer import MultiScorer
import seaborn
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import json
from bson import json_util
from bson.json_util import dumps
%matplotlib inline

In [8]:
thePath = '/Users/gyang/Desktop/ProjectDataScience/data/classify/'

#thePathLut = '/Users/gyang/Desktop/ProjectDataScience/code/'
theCols = os.walk(thePath).__next__()[1]  #use __next__() in py3.x instead of next() in python 2.x
 
theLabels = theCols 

finalWords = list()
theDocs = list()

In [9]:
#this part of code referred by week 3 in class exercise
#no modified

def genCorpus(theText):
    #set dictionaries
    stopWords = set(stopwords.words('english'))
    theStemmer = nltk.stem.porter.PorterStemmer() #Martin Porters celebrated stemming algorithm
    
    #pre-processing
    theText = theText.split()
    tokens = [token.lower() for token in theText] #ensure everything is lower case
    tokens = [re.sub(r'[^a-zA-Z]+', ' ',token) for token in tokens] #remove special characters but leave word in tact
    tokens = [token for token in tokens if token.lower().isalpha()] #ensure everything is a letter
    tokens = [word for word in tokens if word not in stopWords] #rid of stop words
    tokens = [theStemmer.stem(word) for word in tokens] #stem words uing porter stemming algorithm
    tokens = " ".join(tokens) #need to pass string seperated by spaces       

    return tokens

def textToNum(theLabels,thePredLabel):
    theOutLabel = dict()
    cnt = 0
    for word in theLabels:
        theOutLabel[word] = cnt
        cnt = cnt + 1
    return str(theOutLabel[thePredLabel])

for word in theCols:
    cnt = 0
    for file in os.listdir(thePath+word):
        if file.endswith('.txt'):
            try:
                f = open(thePath + word + "/" + file, "r")
                lines = f.readlines()
                lines = [text.strip() for text in lines]
                lines = " ".join(lines)
                finalWords.append(genCorpus(lines))
                theDocs.append(textToNum(theLabels,word) +"_" + str(cnt))
                cnt = cnt +  1
            except:
                pass

In [10]:
vectorizer = TfidfVectorizer(max_features =1000,ngram_range =(1,1))
tdm = pd.DataFrame(vectorizer.fit_transform(finalWords).toarray())

In [11]:
tdm.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.035972,0.00000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000000
1,0.0,0.000000,0.0,0.023988,0.0,0.0,0.000000,0.0,0.000000,0.00000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.136701
2,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.00000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000000
3,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.01908,...,0.0,0.0,0.0,0.039982,0.0,0.0,0.02672,0.0,0.0,0.000000
4,0.0,0.028741,0.0,0.000000,0.0,0.0,0.075636,0.0,0.000000,0.00000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.026406


In [12]:
tdm.columns= vectorizer.get_feature_names()
tdm.index=theDocs

In [13]:
pca = decomposition.PCA(n_components=.95)
pca.fit(tdm)
reducedTDM = pd.DataFrame(pca.transform(tdm)) #reduced tdm distance matrix - features

In [14]:
reducedTDM.index=theDocs

In [15]:
pcaVar = round(sum(pca.explained_variance_ratio_),2)

In [16]:
# labels
fullIndex = reducedTDM.index.values
fullIndex = [int(word.split("_")[0]) for word in fullIndex]

In [17]:
# build models
lr = LogisticRegression() #logistic regression
svm = LinearSVC() #SVM
rfc = RandomForestClassifier() #random forest
dtc = DecisionTreeClassifier(max_depth=30) #decision tree
abdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),learning_rate=1,algorithm="SAMME",n_estimators=300) #ABDT
knn = KNeighborsClassifier() #KNN
bag = BaggingClassifier(bootstrap =True) #bagging
nn = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(15,), random_state=1,alpha=1e-5) #Multi-layer Perceptron classifier

In [18]:
# fit the model
lr_scores = cross_val_score(lr, reducedTDM,fullIndex, cv=10)
svm_scores = cross_val_score(svm, reducedTDM,fullIndex, cv=10)
rfc_scores = cross_val_score(rfc, reducedTDM,fullIndex, cv=10)
dtc_scores = cross_val_score(dtc, reducedTDM,fullIndex, cv=10)
abdt_scores = cross_val_score(abdt, reducedTDM,fullIndex, cv=10)
knn_scores = cross_val_score(knn, reducedTDM,fullIndex, cv=10)
bag_scores = cross_val_score(bag, reducedTDM,fullIndex, cv=10)
nn_scores = cross_val_score(nn, reducedTDM,fullIndex, cv=10)

In [19]:
print("Losgistic Regression Accuracy: %0.8f" % (lr_scores.mean()))
print("SVM Accuracy: %0.8f" % (svm_scores.mean()))
print("Random Forest Accuracy: %0.8f" % (rfc_scores.mean()))
print("Decision Tree Accuracy: %0.8f" % (dtc_scores.mean()))
print("Ada Boost Decision Tree Accuracy: %0.8f" % (abdt_scores.mean()))
print("KNN Accuracy: %0.8f" % (knn_scores.mean()))
print("Bagging Accuracy: %0.8f" % (bag_scores.mean()))
print("Multi-layer Perceptron Accuracy: %0.8f" % (nn_scores.mean()))

Losgistic Regression Accuracy: 0.97307692
SVM Accuracy: 0.95384615
Random Forest Accuracy: 0.88443732
Decision Tree Accuracy: 0.95384615
Ada Boost Decision Tree Accuracy: 0.88461538
KNN Accuracy: 0.96153846
Bagging Accuracy: 0.97307692
Multi-layer Perceptron Accuracy: 0.97307692


In [20]:
#grid search parameters tuning
lr_grid =GridSearchCV(estimator=lr, param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] })
svm_grid = GridSearchCV(estimator=svm,param_grid={"C":[1,0.1,0.05,0.01,0.005]})
rfc_grid =GridSearchCV(estimator=rfc, param_grid={"n_estimators": [10,30,50,100]})
dtc_grid =GridSearchCV(estimator=dtc, param_grid={"max_depth": [10,30,50,100]})
abdt_grid =GridSearchCV(estimator=abdt, param_grid={"n_estimators": [10,30,50,100,200,300]})
knn_grid =GridSearchCV(estimator=knn,param_grid = dict(n_neighbors=range(1,6)))
bag_grid =GridSearchCV(estimator=bag, param_grid={"n_estimators": [10,30,50,100]})
nn_grid =GridSearchCV(estimator=nn, param_grid={"alpha": [1,0.1,0.01,0.001,0.0001,0]})

In [21]:
lr_fit =lr_grid.fit(reducedTDM,fullIndex)
svm_fit =svm_grid.fit(reducedTDM,fullIndex)
rfc_fit =rfc_grid.fit(reducedTDM,fullIndex)
dtc_fit =dtc_grid.fit(reducedTDM,fullIndex)
abdt_fit =abdt_grid.fit(reducedTDM,fullIndex)
knn_fit =knn_grid.fit(reducedTDM,fullIndex)
bag_fit =bag_grid.fit(reducedTDM,fullIndex)
nn_fit =nn_grid.fit(reducedTDM,fullIndex)

In [22]:
def bestscore(grid):
    bestScore = round(grid.best_score_,4)
    parameters = grid.best_params_
    return (bestScore,parameters)

In [23]:
lr_bestscores =bestscore(lr_fit)
svm_bestscores =bestscore(svm_fit)
rfc_bestscores =bestscore(rfc_fit)
dtc_bestscores =bestscore(dtc_fit)
abdt_bestscores =bestscore(abdt_fit)
knn_bestscores =bestscore(knn_fit)
bag_bestscores =bestscore(bag_fit)
nn_bestscores =bestscore(nn_fit)

In [24]:
print("LR --> Best Score: " + str(lr_bestscores[0]) + " and Parameters: " + str(lr_bestscores[1]))
print("SVM --> Best Score: " + str(svm_bestscores[0]) + " and Parameters: " + str(svm_bestscores[1]))
print("Random Forest --> Best Score: " + str(rfc_bestscores[0]) + " and Parameters: " + str(rfc_bestscores[1]))
print("Decision Tree --> Best Score: " + str(dtc_bestscores[0]) + " and Parameters: " + str(dtc_bestscores[1]))
print("ABDT --> Best Score: " + str(abdt_bestscores[0]) + " and Parameters: " + str(abdt_bestscores[1]))
print("KNN --> Best Score: " + str(knn_bestscores[0]) + " and Parameters: " + str(knn_bestscores[1]))
print("Bagging --> Best Score: " + str(bag_bestscores[0]) + " and Parameters: " + str(bag_bestscores[1]))
print("NN --> Best Score: " + str(nn_bestscores[0]) + " and Parameters: " + str(nn_bestscores[1]))

LR --> Best Score: 0.9691 and Parameters: {'C': 1}
SVM --> Best Score: 0.973 and Parameters: {'C': 0.05}
Random Forest --> Best Score: 0.9537 and Parameters: {'n_estimators': 100}
Decision Tree --> Best Score: 0.9653 and Parameters: {'max_depth': 100}
ABDT --> Best Score: 0.4826 and Parameters: {'n_estimators': 10}
KNN --> Best Score: 0.9653 and Parameters: {'n_neighbors': 5}
Bagging --> Best Score: 0.973 and Parameters: {'n_estimators': 30}
NN --> Best Score: 0.9614 and Parameters: {'alpha': 0.0001}


In [25]:
# best model pick up -- Bagging classifier
bestmodel =BaggingClassifier(bootstrap =True,n_estimators =10)
bestmodel.fit(reducedTDM,fullIndex)

BaggingClassifier(base_estimator=None, bootstrap=True,
         bootstrap_features=False, max_features=1.0, max_samples=1.0,
         n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
         verbose=0, warm_start=False)

In [26]:
cross_val_score(bestmodel, reducedTDM,fullIndex, cv=10).mean()

0.96538461538461529

In [36]:
#save best model
filename = 'bestmodel.pkl'
joblib.dump(bestmodel, filename)

['bestmodel.pkl']

In [37]:
# test load the model from disk
best_model = joblib.load(filename)

### Step B: Embed the trained model from above into the twitter streaming engine class, StreamListener.  25-points


In [38]:
import tweepy  
from pymongo import MongoClient
from textwrap import TextWrapper
from tweepy.utils import import_simplejson
json = import_simplejson()

In [39]:
class StreamListener(tweepy.StreamListener):  
    status_wrapper = TextWrapper(width=140, initial_indent='', subsequent_indent='')    
    def on_status(self, status): 
        tempA = self.status_wrapper.fill(status.text)
        tempB = status.retweeted 
        tempC = status.user.lang 
        tempD = status.geo
        
        ############################################################
        #embed code here
        
        testText = list()
        testText.append(genCorpus(self.status_wrapper.fill(status.text)))
        test = vectorizer.transform(testText)
        X2_new = pca.transform(test.toarray())
        x = best_model.predict(X2_new)
        pred_label =theCols[int(x)]
        
        
        ############################################################
        
        if ((("en" in tempC) and (tempB is False)) and (not("RT") in tempA[:2]) and (((("http" or "www") in tempA) and ((' ') in tempA)) or (not("http" or "www") in tempA))):
        #TO DO
            
        #tempA is text body
            try:     
                print(self.status_wrapper.fill(status.text))
                mongo_collection.insert({
                'body': self.status_wrapper.fill(status.text),
                'topic': pred_label,
                'message_id': status.id,
                'screen_name': status.author.screen_name,
                'created_at': status.created_at,
                'followers': status.user.followers_count,
                'friends_count': status.user.friends_count,
                'location': status.user.location
                })
            except Exception as e:  
            #print("HERE")          
                pass 
        
#print("here") 

### Step C: Using your twitter application credentials, perform real-time classification on tweets (~100) that have the following regex set in setTerms and save results to a mongo database/collection: 40-points

In [40]:
#auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
#auth.set_access_token(access_token, access_token_secret)
auth1 = tweepy.auth.OAuthHandler('wNaJv1v48TSxgwS5nsDczvHFt',
                                 'f9YAbBkNtA6aRX5TmVQRZNi7QsSMVz9xjedaleTFNAqIQ4w6Co')  
auth1.set_access_token('915367239733383168-BRMk1IVhls8HjQTZlErdvKpW8m5prPR',
                       'GinBjBKVkP8lqjkfEtfaDKczgawna3yvQ0X6ydGm04w52')  
api = tweepy.API(auth1)

In [41]:
mongo = MongoClient('localhost', 27017)
mongo_db = mongo['twitterDB3s']
mongo_collection = mongo_db['theData']

### Note: Just Use "Stop" to stop running kernel,otherwise it will continue running ...

In [42]:
l = StreamListener()  
streamer = tweepy.Stream(auth=auth1, listener=l, timeout=3000)   
setTerms = ["fishing","hiking","machine learning","mathematics"]
streamer.filter(track =setTerms,languages=['en'])

@tupatsfan Russell fishing for triple double. Teammates being used to throwing out a rebound to give him an assist opportunity. 🙄


/Users/gyang/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:36: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


@BASS_nation The weather is perfect. Should be fishing. Full plate.....shucks!
You know what? It's fine if you just want attention. Looking for sympathy? Ego boost? Fuck'em, go fishing! I'll bite.
Mens Columbia "Gray Ridge" Titanium Fleece Jacket XXL at Tactical Gear Zone -&gt; https://t.co/9KPWhnyapn #columbia #hiking #deals #shopping
…
The BBC will use machine learning to improve programming https://t.co/HDd1dX89Fv
#NewYork #SusquehannaRiver #FlyFishing Report https://t.co/Y4uHagHpLb #NewYorkCity #Buffalo #Rochester #fishing #montanalife #tenkarausa
Guess Arrieta is gonna have to visit Trump at the White House on his own time. Cubs gone fishing.
2017 New Good Quality fishing lure Minnow 70mm/6g 0.5-1 Mt Dive Artificial Ba ... https://t.co/7Xvkm6EzuB https://t.co/R2xGbdyc95
I took this photograph during my visit to Vinh Hy, a fishing village in Vietnam, in July,2014. If you ever travel... https://t.co/CDUs80E7Is
SssnakePit_Lets keep it Real...Itz been a Good Run...Thank U @Cubs for ge

2 Pack Merino Wool Men’s Hiking Socks #fashion #free #shop #style #sales #today #love #giveaway in… https://t.co/3muh7c5glv
Download Fish for Money and earn $15 gift cards just for playing a fishing game! @Apps_that_Pay https://t.co/DywKkQBRL8
Stop fishing for compliments lol it ain’t happening
5 of these little guys this evening in about 30 minutes. Fishing has been tough lately so, for… https://t.co/CEx8i8KmIC
Amazing Girl Fishing em Siem Reap - Cambodia Traditional Fishing - Como pegar peixe (parte 4): https://t.co/kuRyoBiSe7 via @YouTube
Amazing kids fishing video catch a lot of fish by hand - Traditional fis... https://t.co/etpHGJQ0bu via @YouTube
we needa believe it when people say nature has been here before us and will be here after us, we ain’t special https://t.co/fFixqiEZ9o
Secrets to fishing a new lake for big bass from a Yamaha Pro! https://t.co/FfZx1EqsWq RT @Travels_Charley
Great bass fishing insights &amp; advice from Yamaha Pro Ish Monroe https://t.co/nbslAMPrtc RT @Gr

KeyboardInterrupt: 

### Step D: Export your mongo collection from above to a json file called sample.json.  10-points


In [43]:
#extract json to local file
#method 1
c =mongo_collection.find({})

with open('sample.json', 'w') as outfile:
    for line in c:
        json.dump(line,outfile,default =json_util.default)
        outfile.write('\n')   

In [174]:
#method 2 
#json.dump(json_util.dumps(mongo_collection.find({})), open("sample.json", "w"))